Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB10

Use reinforcement learning to devise a tic-tac-toe player.

### Deadlines:

* Submission: Sunday, December 17 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Dies Natalis Solis Invicti ([CET](https://en.wikipedia.org/wiki/Sol_Invictus))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [80]:
from itertools import combinations
from collections import namedtuple, defaultdict
from random import choice
from copy import deepcopy
import functools

from tqdm.auto import tqdm
import numpy as np

In [28]:
MAGIC = [2, 7, 6, 9, 5, 1, 4, 3, 8]
MAGIC_90=[6,1,8,7,5,3,2,9,4]
MAGIC_180=[8,3,4,1,5,9,6,7,2]
MAGIC_270=[4,9,2,3,5,7,8,1,6]
MAGIC_MIRROR=[4,3,8,9,5,1,2,7,6]
MAGIC_MIRROR_90=[2,9,4,7,5,3,6,1,8]
MAGIC_MIRROR_180=[6,7,2,1,5,9,8,3,4]
MAGIC_MIRROR_270=[8,1,6,3,5,7,4,9,2]



In [81]:
from collections import namedtuple

State = namedtuple('State', ['x', 'o'])
def compare(self, other):
    # Confronta i primi set elemento per elemento
    for elem_self, elem_other in zip(self.x, other.x):
        if elem_self < elem_other:
            return -1
        elif elem_self > elem_other:
            return 1

    # Se tutti gli elementi dei primi set sono uguali, confronta i secondi set
    for elem_self, elem_other in zip(self.o, other.o):
        if elem_self < elem_other:
            return -1
        elif elem_self > elem_other:
            return 1

    # Se tutti gli elementi sono uguali, le due istanze sono considerate uguali
    return 0
class CustomState(namedtuple('State', ['x', 'o'])):
    def __eq__(self, other):
        indexes=find_indices(self)
        #print("gli indici sono",indexes)
        for magic in [MAGIC,MAGIC_90,MAGIC_180,MAGIC_270,MAGIC_MIRROR,MAGIC_MIRROR_180,MAGIC_270]:
            
            val =get_values_by_keys(indexes,magic)
            #print(val, other)
            if (sorted(val.x) == sorted(other.x) and sorted(val.o) == sorted(other.o)):
                return True
        
        return False 
    def __hash__(self):
        vett=[]
        indexes=find_indices(self)
        for i,magic in enumerate([MAGIC,MAGIC_90,MAGIC_180,MAGIC_270,MAGIC_MIRROR,MAGIC_MIRROR_180,MAGIC_270]):
            val =get_values_by_keys(indexes,magic)
            #print(i)
            vett.append(val)
        print(vett)
        sorted(vett, key=functools.cmp_to_key())
        print(vett)
        return hash((tuple(vett[0].x), tuple(vett[0].o)))   
   
       

# Esempio di utilizszo:
    

CustomState(set([1,3]),set([5])).__hash__()

x
ritorno da get_values
x
ritorno da get_values
x
ritorno da get_values
x
ritorno da get_values
x
ritorno da get_values
x
ritorno da get_values
x
ritorno da get_values
[CustomState(x={1, 3}, o={5}), CustomState(x={9, 3}, o={5}), CustomState(x={9, 7}, o={5}), CustomState(x={1, 7}, o={5}), CustomState(x={1, 7}, o={5}), CustomState(x={9, 3}, o={5}), CustomState(x={1, 7}, o={5})]


NameError: name 'compare' is not defined

In [68]:
def get_values_by_keys(key_list:CustomState, vector):
    result_values = CustomState(set(),set())
    print("x")
    for key in key_list.x:
            try:
                value = vector[key]
                result_values.x.add(value)
            except IndexError:
                pass
    
    for key in key_list.o:
            try:
                value = vector[key]
                result_values.o.add(value)
            except IndexError:
                pass   
    print("ritorno da get_values")         
    return result_values



In [31]:
def find_indices(pos:State):
    state_tmp=CustomState(set([]),set([]))
    for i,val in enumerate(pos.x):
        #print(val)
        try:
            index = MAGIC.index(val)
            state_tmp.x.add(index)
        except ValueError:
            #print(val)

            print("errore")
            pass
    for i,val in enumerate(pos.o):
        #print(val)
        try:
            index = MAGIC.index(val)
            state_tmp.o.add(index)
        except ValueError:
            print(val)

            print("errore")
            pass
    
    return state_tmp

In [32]:
def print_board(pos):
    """Nicely prints the board"""
    board=""
    for r in range(3):
        board+='\n'

        for c in range(3):
            i = r * 3 + c
            if MAGIC[i] in pos.x:
                print('X', end='')
                board+='X'
            elif MAGIC[i] in pos.o:
                print('O', end='')
                board+='O'
            else:
                print(MAGIC[i], end='')
                board+=str(MAGIC[i])
        print()
    print()
    return board

In [33]:
def win(elements):
    """Checks is elements is winning"""
    return any(sum(c) == 15 for c in combinations(elements, 3))
def block_win_adv(state_adv,pick):
     for c in combinations(state_adv, 2):
         if 15-sum(c) ==pick:
             return 1
     return 0    
def trap_condition(user_state,adversarial_state):
        cnt=0
        for c in combinations(user_state,2):
             val =15-sum(c)

             if val not in adversarial_state and val>0:
                  cnt+=1
                  if cnt>=2:
                       return 0.5         
        return 0    
def state_value(pos: State):
    """Evaluate state: +1 first player wins"""
    if win(pos.x):
        return 1
    elif win(pos.o):
        return -1
    else:
    
        return 0

    

In [34]:
def random_move(available):

    x = choice(list(available))

        
    return x

In [49]:
value_dictionary = {}

hit_state = defaultdict(int)
epsilon = 0.001


for steps in tqdm(range(5000)):
    state = CustomState(set(), set())
    cnt=0
    while (state.x.union(state.o)!=set(range(1,10)) and state_value(state)==0) :
        #chose a random action
        state_tmp=deepcopy(state)
        action = random_move(set(range(1,10))-(state.x.union(state.o)))
        player=cnt%2
        cnt+=1
        #print(cnt)
        if player==1:
            state_tmp.x.add(action)
            reward=state_value(state_tmp)
            if reward==0:
                reward=block_win_adv(state_tmp.o,action)*0.75
            if reward==0:
                reward=trap_condition(state_tmp.x,state_tmp.o)
    
            if state not in value_dictionary:
                value_dictionary[state]={action:0.}
            elif action not in value_dictionary[state]:
                value_dictionary[state][str(action)]=0.

            if state_tmp not in value_dictionary:
                value_dictionary[state_tmp]={action:0.}
            elif action not in value_dictionary[state_tmp]:
                value_dictionary[state_tmp][action]=0.
            value_dictionary[state][action]=(1-0.1)*value_dictionary[state][action]+0.1*(reward+0.7* max(value_dictionary[state_tmp].values()))
            state=deepcopy(state_tmp)
        
        else:
            state_tmp.o.add(action)
            reward=state_value(state_tmp)
            if reward==0:
                reward=-block_win_adv(state_tmp.x,action)*0.75
            if reward==0:
                reward=-trap_condition(state_tmp.o,state_tmp.x)
            if state not in value_dictionary:
                value_dictionary[state]={action:0.}
            elif action not in value_dictionary[state]:
                value_dictionary[state][action]=0.

            if state_tmp not in value_dictionary:
                value_dictionary[state_tmp]={action:0.}
            elif action not in value_dictionary[str(state_tmp)]:
                value_dictionary[state_tmp][action]=0.

            
            value_dictionary[state][action]=(1-0.1)*value_dictionary[state][action]+0.1*(reward+0.7* min(value_dictionary[state_tmp].values()))
            
            state=deepcopy(state_tmp)
        

  0%|          | 0/5000 [00:00<?, ?it/s]

RecursionError: maximum recursion depth exceeded while calling a Python object

In [42]:
def stampa_dizionario(dizionario, livello=0):
    spazi = "  " * livello
    for chiave, valore in dizionario.items():
        if isinstance(valore, dict):
            print(f"{spazi}{chiave}:")
            stampa_dizionario(valore, livello + 1)
        else:
            print(f"{spazi}{chiave}: {valore}")



stampa_dizionario(value_dictionary)


CustomState(x=set(), o=set()):
  4: 0.0
CustomState(x=set(), o={4}):
  4: 0.0
  1: 0.0
CustomState(x={1}, o={4}):
  1: 0.0


In [37]:
import time
def play_against_him():
    print_board(State(set(),set()))
    state=State(set(),set())
    while (state.x.union(state.o)!=set(range(1,10)) and state_value(state)==0) :
        list_move = sorted(value_dictionary[str(state)], key=value_dictionary[str(state)].get)
        for i in range(len(list_move)):
            val=list_move[i]
            if val not in state.x:
                break
        state.x.add(val)
        print_board(state)
        time.sleep(1)

        numero_inserito = input("Make your move")

        try:
            #print(board+"\n")
            numero_inserito = int(numero_inserito)

            if(numero_inserito not in (set(range(1,10))-state.x.union(state.o))):
                print("mossa non valida")
                break
            else:

                state.o.add(numero_inserito)
            print_board(state)
        except ValueError:
            print("Not valid.")()
    if(state_value(state)==1):
        print("RL win")
    elif(state_value(state)==-1):
        print("You win ")
    else:
        print("pair")


play_against_him()



276
951
438



KeyError: 'State(x=set(), o=set())'

In [ ]:
def play_against_random():
    print_board(State(set(),set()))
    state=State(set(),set())
    print("X---->RL")
    print("O----->RANDOM")
    while (state.x.union(state.o)!=set(range(1,10)) and state_value(state)==0) :
        list_move = sorted(value_dictionary[str(state)], key=value_dictionary[str(state)].get)
        for i in range(len(list_move)):
            val=list_move[i]
            if val not in state.x:
                break
        state.x.add(val)
        
        move=random_move(set(range(1,10))-(state.x.union(state.o)))
        state.o.add(move)
        print_board(state)

play_against_random()        

        



276
951
438

X---->RL
O----->RANDOM
27O
9X1
438

OXO
9X1
438

OXO
OX1
4X8



In [ ]:
def is_symmetric(move1, move2):
    # Definisci la mappa di simmetria rispetto a una rotazione di 180 gradi
    symmetry_map = {'1': '9', '2': '8', '3': '7', '4': '6', '5': '5', '6': '4', '7': '3', '8': '2', '9': '1'}

    # Verifica se le mosse sono simmetriche rispetto a una rotazione di 180 gradi
    return all(symmetry_map.get(digit) == move2[i] for i, digit in enumerate(move1))

# Esempio di utilizzo:
move_user1 = '27'
move_user2 = '59'

if is_symmetric(move_user1, move_user2):
    print("Le mosse sono simmetriche rispetto a una rotazione di 180 gradi.")
else:
    print("Le mosse non sono simmetriche rispetto a una rotazione di 180 gradi.")


In [ ]:
fruits = ['apple', 'banana', 'orange']

for index, fruit in enumerate(fruits, start=1):
    print(f"Index: {index}, Fruit: {fruit}")


Index: 1, Fruit: apple
Index: 2, Fruit: banana
Index: 3, Fruit: orange
